In [1]:
# findspark ...

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pv_diff_app").getOrCreate() 
spark

In [3]:
pvdf = spark.read.csv('pv_sample_input_1.csv',inferSchema=True,header=True)

In [4]:
import pandas as pd
pd.options.display.max_rows = None
pd.options.display.max_columns = None 
pd.options.display.max_colwidth = None
pd.options.display.precision = 2

# as in https://towardsdatascience.com/8-commonly-used-pandas-display-options-you-should-know-a832365efa95

In [5]:
pvdf.printSchema()

root
 |-- pnl report: string (nullable = true)
 |-- report ccy: string (nullable = true)
 |-- v: string (nullable = true)
 |-- deal id: integer (nullable = true)
 |-- product: string (nullable = true)
 |-- deal ccy: string (nullable = true)
 |-- PV: double (nullable = true)



In [6]:
pvdf = pvdf.where(pvdf['report ccy'] == 'CCY')
# as in https://sparkbyexamples.com/pyspark/pyspark-where-filter/

In [7]:
pvdf.toPandas()

,pnl report,report ccy,v,deal id,product,deal ccy,PV
0,LDN desk CCY,CCY,v1,100,irs,GPB,1023.21
1,LDN desk CCY,CCY,v1,101,irs,USD,1012.54
2,SYD desk CCY,CCY,v1,202,repo,AUD,1012.98
3,NY desk CCY,CCY,v1,303,sfut,EUR,1022.15
4,LDN desk CCY,CCY,v2,100,irs,GPB,1023.76
5,LDN desk CCY,CCY,v2,102,irs,NZD,1021.24
6,SG desk CCY,CCY,v2,202,repo,AUD,1000.34
7,NY desk CCY,CCY,v2,303,sfut,EUR,1003.74


In [8]:
pvdf.groupBy('pnl report').pivot("v").sum("PV").toPandas()
# as in https://sparkbyexamples.com/pyspark/pyspark-pivot-and-unpivot-dataframe/

,pnl report,v1,v2
0,NY desk CCY,1022.15,1003.74
1,SG desk CCY,NaN,1000.34
2,SYD desk CCY,1012.98,NaN
3,LDN desk CCY,2035.75,2045.00


In [14]:
scope_pddf = pd.DataFrame({
                    'product': ['irs','fra','lfut','repo'],
                    'take': [True,False,False,False]
                    })
 
scope_psdf = spark.createDataFrame(scope_pddf)

# as in https://www.geeksforgeeks.org/creating-a-pyspark-dataframe/ 

In [15]:
scope_psdf = scope_psdf.where(scope_psdf['take'])

In [17]:
scope_psdf.show()

+-------+----+
|product|take|
+-------+----+
|    irs|true|
+-------+----+



In [18]:
pvdf.join(other=scope_psdf,on='product',how='inner').groupBy('pnl report').pivot("v").sum("PV").toPandas()

,pnl report,v1,v2
0,LDN desk CCY,2035.75,2045.0
